# [SOC-88] Mapping Crime in San Francisco

### Professor David Harding

## Table of Contents

[Introduction](#intro)

[The Data](#data)

[Base Maps](#1)
   - [Question 1](#q1)

[Markers](#2)
   - [Question 2](#q2)
   - [Question 3](#q3)
   - [Question 4](#q4)
    
[Choropleth Maps](#3)
   - [Question 5](#q5)
   - [Question 6](#q6)



## Introduction <a id='intro'></a>

In this homework, you will practice different data mapping techniques you learned about in lecture and lab. The data has been taken from [SF Data](https://data.sfgov.org/), San Francisco's open data site. 

There are two main data files used in this assignment: **SFPD_incidents_2016.csv** and **sfpd-police-districts.geojson**. 

The first file, **SFPD_incidents_2016.csv**, has records of all police incidents that took place in 2016. Its columns contain information such as the latitude-longitude information of incidents, police precinct and neighborhood in which the incident occurred, time and date of the report, type of crime, etc. 

The second file, **sfpd-police-districts.geojson**, contains geographic information about the boundaries of San Francisco police districts. These boundaries are necessary for making choropleth plots.

---


We will begin by running a code cell that will load the libraries you'll be using.

In [1]:
# load the necessary software
from datascience import *
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
import folium
import json

## The Data <a id='data'></a>


Our main dataset comes from the [city of San Francisco's open data portal](https://data.sfgov.org/Public-Safety/Police-Department-Incident-Reports-Historical-2003/tmnf-yvry). 

Run the next cell to load the incident data.

In [2]:
# load SF Police Incident Data, 2016
incidents = Table().read_table('data/SFPD_incidents_2016.csv')
incidents.show(5)

Each row in this table represents a different incident reported to the San Francisco Police Department (SFPD). Most of the columns are fairly intuitive, but we'll break down a few of particular interest:

- `IncidntNum`and `PdId` are kinds of identifiers for each incident, used for organization within the police department.

- `Category` classifies the incident as on of 37 types. We can see all possible categories using the `group` method.

In [3]:
# show the unique categories
incidents.group("Category").show()



- `Descript` gives more information on what occurred during the incident. You can think of the `Descript` as a subtype of the category in `Category`. There are too many unique descriptions to list them all; it makes more sense to select a particular category and then list the possible descriptions for only that category. In the next cell, you can view the possible `Descript` values for incidents falling under the category `"NON-CRIMINAL"`.

In [4]:
# show the unique incident descriptions for the NON-CRIMINAL category
incidents.where("Category", "NON-CRIMINAL").group('Descript').show()

- `Resolution` gives information about what the police did about the incident. Once again, we can view all possible resolution options using the `group` method

In [5]:
# show the unique resolutions
incidents.group("Resolution").show()

- Finally, `X`, `Y`, and `Location` give geographic data about the incident. `X` represents the longitude, `Y` represents the latitude, and `Location` has both the latitude and longitude together.

## Base Maps <a id='1'></a>

### Question 1 <a id='q1'></a>
Create a base map centered on San Francisco. Choose an appropriate zoom start and tiles.

*Note: we're going to be creating several maps in this homework, so it's easier to create variables for the starting coordinates, zoom, and tiles, and use them over and over again, rather than rewrite them in every map we make.*

In [6]:
# add coordinates for San Francisco
sf_coordinates = [incidents.column('Y').item(0), incidents.column('X').item(0)]
sf_zoom_start = 11
sf_tiles = "OpenStreetMap"

# create a map of San Francisco
sf_map = folium.Map(location=sf_coordinates, zoom_start=sf_zoom_start, tiles=sf_tiles)
sf_map

## Markers <a id='2'></a>

In the next two cells, we've isolated two police incidents for you.

In [7]:
incidentA = incidents.where("PdId", 16009503010045)
incidentA

IncidntNum | Category     | Descript                              | DayOfWeek | Date       | Time  | PdDistrict | Resolution     | Address                | X        | Y       | Location                              | PdId
160095030  | EMBEZZLEMENT | EMBEZZLEMENT, GRAND THEFT BY EMPLOYEE | Monday    | 02/01/2016 | 18:05 | SOUTHERN   | ARREST, BOOKED | 800 Block of MARKET ST | -122.408 | 37.7842 | (37.7841893501425, -122.407633520742) | 16009503010045

In [8]:
incidentB = incidents.where("PdId", 16014933626039)
incidentB

IncidntNum | Category | Descript                 | DayOfWeek | Date       | Time  | PdDistrict | Resolution | Address              | X        | Y       | Location                              | PdId
160149336  | ARSON    | FIRE, UNLAWFULLY CAUSING | Friday    | 02/19/2016 | 21:42 | RICHMOND   | NONE       | 100 Block of 28TH AV | -122.488 | 37.7859 | (37.7859188368234, -122.488216032172) | 16014933626039

### Question 2 <a id='q2'></a>
Create a marker for each of the above incidents, including:
* incident location
* an appropriate and informative pop-up (appears when you hover over the marker)
* an appropriate and informative tooltip (appears when you click on the marker)
* an appropriate color and type for the icon, given the type of incident

Hint: you can view the list of icon options at https://getbootstrap.com/docs/3.3/components/

In [22]:
# a clean map for the markers
marker_map = folium.Map(location=sf_coordinates, zoom_start=sf_zoom_start, tiles=sf_tiles)

# For Incident A
coordinateA = [incidentA.column('Y'), incidentA.column('X')]
popupA = incidentA.column("Descript")
tooltipA = incidentA.column("Category")
folium.Marker(location=coordinateA, popup=popupA, 
              tooltip=tooltipA, icon=folium.Icon(color="cadetblue", icon="exclamation-sign")).add_to(marker_map)

# view the map
marker_map

In [23]:
# For Incident B
coordinateB = [incidentB.column('Y'), incidentB.column('X')]
popupB = incidentB.column("Descript")
tooltipB = incidentB.column("Category")
folium.Marker(location=coordinateB, popup=popupB, 
              tooltip=tooltipB, icon=folium.Icon(color="red", icon="exclamation-sign")).add_to(marker_map)
marker_map

Next, we'd like to map all incidences of disorderly conduct.

First, we make a table that only contains disorderly conduct incidents.

In [18]:
# filter for just disorderly conduct
disorderly = incidents.where("Category", "DISORDERLY CONDUCT")
disorderly.show(50)

### Question 3 <a id='q3'></a>
Fill in the code below to create markers for all disorderly conduct incidents.

As in question 2, choose the appropriate coordinates, popup, tooltip, color, and icon for the type of incident.

In [16]:
# create a clean map for the disorderly conduct incidents
disorderly_map = folium.Map(location=sf_coordinates, tiles=sf_tiles, zoom_start=sf_zoom_start)

# make a marker for each disorderly conduct incident
for i in range(disorderly.num_rows):
    incidentC = disorderly.take(i)
    coordinateC = [incidentC.column('Y'), incidentC.column('X')]
    popupC = incidentC.column("Descript")
    tooltipC = incidentC.column("Category")
    folium.Marker(location=coordinateC, popup=popupC, 
                  tooltip=tooltipC, icon=folium.Icon(color="red", icon="exclamation-sign")).add_to(disorderly_map)
    
# show the map
disorderly_map

### Question 4 <a id='q4'></a>

Describe the features you chose for questions 1, 2, and 3, including:
* map tiles
* marker icon
* marker color
* marker popup and tooltip

Why were those features good for the data in those questions?

For question 1, the feature "openstreetmap" for map tiles was a good tool since it allows for viewers to notice exactly how SF is organized by displaying the streets. For question 2, the same map tile was used to display a descriptive map of SF streets. The marker icon was used to display the severity of the incident. "Red" was a good indication to demonstrate the severity of "fire" when comparing the the marker color "cadetblue" for "embezzlement." Both marker icons have an exclamation sign since they are categorized as "disorderly conduct". The marker pop ups are good short desciptions of what the incident entails whereas the marker tooltip gives a more thorough description of the incident of disprder;y conduct. For question 3, I chose the "OpenStreetMap" tiles since it is visually appealing and descriptive visual of SF. The marker icon is an "exclamation-sign" since the incidents are categorized as disorderly conduct and therefore have a red marker color. The marker popup provides a brief description of the incident while the tooltip goes more in depth about each disorderly conduct.

## Choropleth maps <a id='3'></a>

In this section, you're going to create a choropleth map with the number of non-criminal mental health related incidents in each district.

First, we need to load the geojson file that gives the boundaries for the police districts. Run the next cell to load the geojson.

In [24]:
# load SFPD district boundaries
sf_districts = json.load(open('data/sf-police-districts.geojson'))
sf_districts

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'shape_area': '91344142.1925',
    'shape_leng': '87550.2751419',
    'company': 'B',
    'shape_le_1': '100231.353916',
    'district': 'SOUTHERN'},
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[-122.39186138812263, 37.79424680970543],
       [-122.39178965045961, 37.7941712252344],
       [-122.39173396937666, 37.794202997266886],
       [-122.39166681055336, 37.794132183335286],
       [-122.39172264479394, 37.79410052824544],
       [-122.3916726266299, 37.79404791882388],
       [-122.39198184041989, 37.79387175411522],
       [-122.39158906053727, 37.7935276602537],
       [-122.391723729351, 37.79342884992162],
       [-122.39081210865645, 37.79266254752923],
       [-122.38929874837604, 37.791390503319754],
       [-122.38880784444565, 37.790977844242896],
       [-122.38893019202212, 37.790880005297964],
       [-122.38861191205169, 37.790313282442106],
       [-122.3880494246161

You can see the districts overlaid onto the San Francisco Map by running the next cell.

In [25]:
# make the folium geojson object and add to a map of SF
m = folium.Map(sf_coordinates, zoom_start=sf_zoom_start, tiles=sf_tiles)

folium.GeoJson(
    sf_districts,
    style_function=lambda feature: {
        'fillColor': 'white',
        'color': 'blue',
        'weight': 2,
        'dashArray': '5, 5'
    }
).add_to(m)
m

To make our choropleth overlay, we must first get the counts of mental health incidents by district. First, we use the `where` method to select only the incidents that have a description of "AIDED CASE, MENTAL DISTURBED".

In [26]:
# mental health related incidents
mental_health = incidents.where("Descript", are.equal_to("AIDED CASE, MENTAL DISTURBED"))
mental_health.show(3)

Next, we use `group` to get the counts of mental health incidents per police district.

In [27]:
# get the counts of mental health incidents by district
mental_health_by_district = mental_health.group("PdDistrict")
mental_health_by_district

PdDistrict | count
BAYVIEW    | 40
CENTRAL    | 70
INGLESIDE  | 52
MISSION    | 103
NORTHERN   | 75
PARK       | 35
RICHMOND   | 44
SOUTHERN   | 143
TARAVAL    | 49
TENDERLOIN | 79

And finally, we convert the counts to a DataFrame so that it works with the Folium software.

In [28]:
# convert to DataFrame
mental_health_df = mental_health_by_district.to_df()
mental_health_df.head()

PdDistrict  count
0    BAYVIEW     40
1    CENTRAL     70
2  INGLESIDE     52
3    MISSION    103
4   NORTHERN     75

### Question 5 <a id='q5'></a>
Complete the following code to create a choropleth overlay showing the counts of mental health incidents for each police district. Choose an appropriate and informative:
* fill color (using a colormap)
* fill opacity
* legend name

Hint: you can find colormap options at https://matplotlib.org/gallery/color/colormap_reference.html

In [35]:
# create a clean map for the choropleth
m = folium.Map(sf_coordinates, zoom_start=sf_zoom_start, tiles=sf_tiles)

# create the choropleth overlay
folium.Choropleth(
    geo_data=sf_districts, #geojson data
    data=mental_health_df, #dataframe of counts
    columns=['PdDistrict', 'count'], #info in data dataframe
    key_on='feature.properties.district', #info in json
    fill_color='YlOrRd',
    fill_opacity=0.7,
    legend_name= "Number of Mental Health Incidents"
).add_to(m)
m

### Question 6 <a id='q6'></a>
Explain your design choices for the choropleth map. What options did you consider? What options did you end up choosing, and why? Be sure to reference the context of the data when you explain your choices.

For the fill color I decided to choose 'YlOrRd' since it vividly demonstrates the difference in the number of mental health incidents by area, with red being very high. This color could be alarming to viewers since it can demonstrate the need to take action on the high number of mental health incidents in the red areas of SF. The opacity was 0.7 in order to demonstrate the SF area that the choropleth map is representing.

----


Data Science Modules: http://data.berkeley.edu/education/modules

Data Science Offerings at Berkeley: https://data.berkeley.edu/academics/undergraduate-programs/data-science-offerings



Notebook developed by: Keeley Takimoto